# End to End project on Twitter LiveStreaming, Numpy, Pandas DataFrame, NLP, Sentiment Analysis, Network Analysis,
Geographical locatio, creating twitter map

Why Analyze twitter data?

There are millions of tweets created everyday from across the entire world, in many different languages.

In this this course we are going to:
1.	Collect twitter data
2.	How to process text
3.	How to analyze twitter networks and,
4.	How to map twitter data geographically 

Twitter is one of the largest social networking sites in operation right now. While Twitter is far from a comprehensive record of the public conversation, it can help to provide insight into popular trends and important cultural and political moments. If you are a data scientist in industry, analyzing Twitter can be used for task such as:

1.	Marketing or product analysis 
2.	Measure of public opinion on important political or social topics

Twitter data has been used to analyze political polarization, public opinion of world leader, and the spread of protest movements. Of course, you can’t access all of what happens on Twitter. It may seem obvious to say, but you can only collect information on what people say, not who is watching passively. 

If you’re looking to use free tools for doing your analysis, you are also limited in two important ways: 
1.	You cannot collect data from past. If you wanted data on your company from one year ago and you didn’t collect, it at the time you are out of luck.
2.	Twitter only offers a sample of their data for free, what they say is a 1 % sample. However, a 1 % sample of Twitter is still on the order of a few million tweets a day.

In each of those tweets, you get a lot of information: the text of the tweet, user profile – like the number of followers and followees the user has – geolocation and other extras. You can also get all of those information from retweets and quoted tweets.

Use Case 1:

You have been asked to identify the success (or failure) of a particular product. You can use Twitter analysis strategy such as following to  to best executed this:

1.	Collect mentions of the product and identify if people are talking about it positively.
2.	Examine the size of the retweet network mentioning the product.
3.	Analyzing the geographical penetration of user mentioning the product.

Collecting data through the Twitter API

By now you might have a good idea of what kind of insights can be gained through Twitter data, let’s look at how to collect some Twitter data ourselves through the Twitter API.

API : Application Programming interface
	It is a method of accessing data from business or government organization.

Many, if not most social media companies have some kind of API which are made available to third-party developers and researchers. Twitter has multiple APIs which can be used for different purposes. These change from timt-to-time and Twitter sometimes adds or removes APIs.

This includes:
1.	Search API: Which allows access to tweets from the past week,
2.	Ads API: Which focuses on Twitter ads
3.	Straming API: The Streaming API allows us to collect a sample of tweets in real-time based on keywords, user IDs, and locations. The connections stays open until you close it.


In this section we will focus on the streaming API.

The Steaming API has two endpoints, filter and sample.
1.	Filter endpoint: With the filter endpoint, you can request data on a few hundred keywords, a few thousand usernames, and 25 location ranges.
2.	Sample endpoint: With the sample endpoint, Twitter will return a 1 % Random sample of all of the twitter.

To collect data from the Streaming API, we are going to use package called ‘tweepy’. It abstracts away much of the work we need to set up a stable Twitter Streaming API connection. When you do this in practice, you’re going to have to set up your own Twitter account and API keys for authentication.

from tweepy.streaming import StreamListener import time 
class SListener(StreamListener): 
def __init__(self, api = None): 
self.output = open('tweets_%s.json' % time.strftime('%Y%m%d-%H%M%S'), 'w') 
self.api = api or API() 

Our ‘SListener’ object inherits from a general ‘StreamListener’ class included with ‘tweepy’. It opens a new timestamped file in which to store tweets and takes an optional API argument.

We first must authenticate with Twitter.

tweepy aunthentication

from tweepy import OAuthHandler
from tweepy import API

auth = OAuthHandler (consumer_key, consumer_secrete)

auth.set_access_token ( access_token, access_token_secret)

api = API(auth)

from tweepy import Stream

listen = SListener(api)

stream = Steam(auth, listen)

stream.sample()

OAuthentication, the authentication protocol which the Twitter API uses, requires four tokens which we obtain from the Twitter developer site: the consumer key and consumer secret, and the access token and access token secret.

Then we set the access token and the access token secret

Finally, we pass the auth object to the tweepy API object.

Now we can get the collecting data.

If we were going to take a random sample of all of Twitter, we would use the ‘sample’ endpoint

Lastly, we use sample collection method to collect data.


In [ ]:
# Setting up tweepy authentication

In [ ]:
consumer_key = 'Your customer_key'
consumer_secret = 'Your_Customer_Secret'
access_token = 'Your_access_token'
access_token_secret ='Your_token_secret'

In [6]:
from tweepy import OAuthHandler
from tweepy import API

# Consumer key authentication
auth = OAuthHandler(consumer_key, consumer_secret)

# Access key authentication
auth.set_access_token(access_token, access_token_secret)

# Set up the API with the authentication handler
api = API(auth)

# Collecting data on keywords

Now that we've set up the authentication, we can begin to collect Twitter data. 
Recall that with the Streaming API, we will be collecting real-time Twitter data 
based on either a sample or filtered by a keyword.

In our example, we will collect data on any tweet mentioning #rstats or #python in 
the tweet text, username, or user description with the filter endpoint.

In [ ]:
class MyListener(tweepy.StreamListener):

    def on_data(self, data):
        try:
            with open('tweet_stream.json', 'a') as file:
                file.write(data)
                print(data)
                return True
        except BaseException as e:
            print("Error on_data: {}".format(str(e)))
        return True


twitter_stream = tweepy.Stream(auth, MyListener())
twitter_stream.filter(track=['suman ale'])

In [8]:
from tweepy.streaming import StreamListener
import json
import time
import sys

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

In [ ]:
from tweepy import Stream

# Set up words to track
keywords_to_track = ['#rstats', '#python']

# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data
stream.filter(track = keywords_to_track)

# Understanding Twitter JSON

Now that you’ve collected some Twitter data, lets dig into the structure of the data. The data is returned in Javascrpt Objhect Notation or JSON. JSON is a special data format which is both human-readable and is easily transferred between machines. JSON is structured a lot like Python Objects and is composed of a combination of dictionaries and lists. 

Twitter JSON contains several important child JSON objects. These are like dictionaries stored in other dictionaries. The important one which we cover in this file are ‘user’, ‘place’, and ‘extended_tweet’.

‘user’ contains all the useful information you would want to know about the user who tweeted including their name, their Twitter handle, their Twitter bio, their location, and if they’re verified.

‘place’ contains geolocation.

‘extended_tweet’ contains over 140 characters.

When a tweet is a retweet or contain a quoted tweet, the whole of that tweet will be contained with the Twitter JSON. For retweets, that tweet will be stored in ‘retweeted_status’, and for quoted tweet in ‘quoted_status’.

Loading and Accessing JSON tweets

import json

tweet_json = open(‘tweet-example.json’, ‘r’).read()

tweet = json.loads(tweet_json)

tweet[‘text’]


Then, we’ll used the json package and the loads methods to convert the JSON into a Python dictionary. Lastly, we access the value of interest by using its appropriate key.

Child Twitter JSON can be accessed as nested dictionaries.

tweet[‘user’][‘screen_name’]

tweet[‘user’][‘name’]

tweet[‘user’][‘created_at’]


In [ ]:
# Load JSON
import json

# Convert from JSON to Python object
tweet = json.loads(tweet_json)

# Print tweet text
print(tweet['text'])

# Print tweet id
print(tweet['id'])

# Accessing user data

Much of the data which we want to know about the Twitter data is stored in child JSON objects. 
We will access several parts of the user's information with the user child JSON object.

In [ ]:
# Print user handle
print(tweet['user']['screen_name'])

# Print user follower count
print(tweet['user']['followers_count'])

# Print user location
print(tweet['user']['location'])

# Print user description
print(tweet['user']['description'])

# Accessing retweet data

In [ ]:
# Print the text of the tweet
print(rt['text'])

# Print the text of tweet which has been retweeted
print(rt['retweeted_status']['text'])

# Print the user handle of the tweet
print(rt['user']['screen_name'])

# Print the user handle of the tweet which has been retweeted
print(rt['retweeted_status']['user']['screen_name'])

# Processing Twitter text

Arguably, the most important part of the tweet is the text. Recall the primary location of the text in the Twitter JSON can be accessed via the ‘text’ key. However, that’s not the only place where we can extract meaningful text-based elements. Twitter has made it possible to use more than 140 characters. For tweets which are over 140 characters, we can access the text through the ‘extended_tweet’ child JSON object and the ‘full_text’ field.

Also recall, to get those objects in retweets and quoted tweets, we have to access the ‘retweeted_status’ or ‘quoted_status’ child JSON objects. Once we parse the JSON, we can access the elements by chaining together dictionaries.

Flattening Twitter JSON

Extended_tweet[‘extended_tweet-full_text’] = extended_tweet[‘extended_tweet’][‘full_text’]

To analyze tweets at scale, it’s helpful to put everything into a pandas DataFrame. This allows us to apply certain analysis methods across all rows and multiple columns. However, with the multiple JSON children, we cannot access values in those children easily in columns. To do this, we flatten the JSON by storing the children values in top-level keys. For convenience, we separate the original keys with a dash. 

To put these all into a DataFrame:
1.	we will have to loop through all of the tweets individually.
2.	We open a JSON file full of tweets. We’ll then split the JSON by the newline character. 
3.	Then for each tweet we parse the JSON with ‘json.loads’, which converts JSON to Python
4.	We check if the tweet contains a field of interet, such as 280 character tweet in ‘extended_tweet’ if it does, we created a top-level field for it in the tweet dictionary.
5.	Then add this tweet object to a list of tweets
6.	This is now a list of dictionaries which represent tweets.

Because this is a list of dictionaries we can pass it as a argument to the pandas.


In [ ]:
# Print the tweet text
print(quoted_tweet['text'])

# Print the quoted tweet text
print(quoted_tweet['quoted_status']['text'])

# Print the quoted tweet's extended (140+) text
print(quoted_tweet['quoted_status']['extended_tweet']['full_text'])

# Print the quoted user location
print(quoted_tweet['quoted_status']['user']['location'])

In [ ]:
# Store the user screen_name in 'user-screen_name'
quoted_tweet['user-screen_name'] = quoted_tweet['user']['screen_name']

# Store the quoted_status text in 'quoted_status-text'
quoted_tweet['quoted_status-text'] = quoted_tweet['quoted_status']['text']

# Store the quoted tweet's extended (140+) text in 
# 'quoted_status-extended_tweet-full_text'
quoted_tweet['quoted_status-extended_tweet-full_text'] = quoted_tweet['quoted_status']['extended_tweet']['full_text']

# A tweet flattening function

In [ ]:
def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    
    # Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)
    
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
    
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = tweet_obj['entended_tweet']['full_text']
    
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']
            
        tweets_list.append(tweet_obj)
    return tweets_list

# Loading tweets into a DataFrame

Now it's time to import data into a pandas DataFrame so we can analyze tweets at scale.

We will work with a dataset of tweets which contain the hashtag '#rstats' or '#python'. 
This dataset is stored as a list of tweet JSON objects in data_science_json.

In [ ]:
# Import pandas
import pandas as pd

# Flatten the tweets and store in `tweets`
tweets = flatten_tweets(data_science_json)

# Create a DataFrame from `tweets`
ds_tweets = pd.DataFrame(tweets)

# Print out the first 5 tweets from this dataset
print(ds_tweets['text'].values[0:5])

# Counting Words
Now that we’re able to analyze tweets at scale, it’s time to begin processing tweet text. 
One of the most basic analyses we can perform is counting the number of times a word or a phrase has appeared in text, and how many times it has appeared compared to other keywords in the text. 
Counting words is the most basic step we can take in automating the analysis of text. 
It allows us to convert words into numbers. 
In practice, counting words can tell us how many times a company, product, or hashtag is mentioned.

Counting with str.contains
To count the frequency of a particular keywords we’ll use ‘str.contains’ . 
This is a string method for the pandas Series object which tells us whether or not a row contains the keyword in question. 
‘str.contains’ return Boolean values. It takes keyword argument case. Setting case = Fasle will make it case insensitive.


In [ ]:
# Flatten the tweets and store them
flat_tweets = flatten_tweets(data_science_json)

# Convert to DataFrame
ds_tweets = pd.DataFrame(flat_tweets)

# Find mentions of #python in 'text'
python = ds_tweets['text'].str.contains('#python', case = False)

# Print proportion of tweets mentioning #python
print("Proportion of #python tweets:", np.sum(python) / ds_tweets.shape[0])

# Looking for text in all the wrong places

Recall that relevant text may not only be in the main text field of the tweet. 
It may also be in the extended_tweet, the retweeted_status, or the quoted_status. 
We need to check all of these fields to make sure we've accounted for all the of the relevant text. 
We'll do this often so we're going to create a function which does this.

In [ ]:
def check_word_in_tweet(word, data):
    """Checks if a word is in a Twitter dataset's text. 
    Checks text and extended tweet (140+ character tweets) for tweets,
    retweets and quoted tweets.
    Returns a logical pandas Series.
    """
    contains_column = data['text'].str.contains(word, case = False)
    contains_column |= data['extended_tweet-full_text'].str.contains(word, case = False)
    contains_column |= data['quoted_status-text'].str.contains(word, case = False) 
    contains_column |= data['quoted_status-extended_tweet-full_text'].str.contains(word, case = False) 
    contains_column |= data['retweeted_status-text'].str.contains(word, case = False) 
    contains_column |= data['retweeted_status-extended_tweet-full_text'].str.contains(word, case = False)
    return contains_column

# Comparing #python to #rstats

Now that we have a function to check whether or not the word is in the tweet in multiple places, 
we can deploy this across multiple words and compare them. 
Let's return to our example with the data science hashtag dataset. 
We want to see how many times that #rstats occurs compared to #python.

In [ ]:
# Find mentions of #python in all text fields
python = check_word_in_tweet('#python', ds_tweets)

# Find mentions of #rstats in all text fields
rstats = check_word_in_tweet('#rstats', ds_tweets)

# Print proportion of tweets mentioning #python
print("Proportion of #python tweets:", np.sum(python) / ds_tweets.shape[0])

# Print proportion of tweets mentioning #rstats
print("Proportion of #rstats tweets:", np.sum(rstats) / ds_tweets.shape[0])

# Creating time series data frame

Time series data is used when we want to analyze or explore variation over time. 
This is useful when exploring Twitter text data if we want to track the prevalence of a word or set of words.

The first step in doing this is converting the DataFrame into a format which can be 
handled using pandas time series methods. 
That can be done by converting the index to a datetime type.

Time Series

Tweets about companies, product, and political issues vary by the day, hour, minute, and even to the second. We want to be able to capture that variation over time. When data is tagged with a date and time, time, this is known as ‘time series’ or ‘over time’ data.

We can convert the Twitter format for date. Panda is smart enough to create date time stored in ‘created_at’ – to a date time type with the ‘to_datetime’ method.

tweets[‘created_at] = pd.to_datetime(tweets[‘created_at’])

Next, we set the index of the DataFrame to the ‘created_at’ column using the ‘set_index’ method

tweets = tweets.set_index(‘created_at’)


We first generate a summary statistics over the metric we’re interested in. We can use Series method ‘resample’ for this purpose. ‘resample’ allows us to summarize over a time window of our choice and apply a function to it. We will use ‘resample’ with the ‘mean’ method to generate averages over one-minute windows.

Lastly we will plot those keyword means over time.


# Time series data is used when we want to analyze or explore variation over time. 
This is useful when exploring Twitter text data if we want to track the prevalence of a word or set of words.

The first step in doing this is converting the DataFrame into a format which can 
be handled using pandas time series methods. That can be done by converting the index to a datetime type.

In [ ]:
# Print created_at to see the original format of datetime in Twitter data
print(ds_tweets['created_at'].head(5))

# Convert the created_at column to np.datetime object
ds_tweets['created_at'] = pd.to_datetime(ds_tweets['created_at'])

# Print created_at to see new format
print(ds_tweets['created_at'].head(5))

# Set the index of ds_tweets to created_at
ds_tweets = ds_tweets.set_index('created_at')

In [ ]:
# Generating mean frequency
We need to produce a metric which can be graphed over time. 
Our function check_word_in_tweet() returns a boolean Series. Remember that the boolean value True == 1, 
so we can produce a column for each keyword we're interested in and use it to understand its over time prevalence.

In [ ]:
# Create a python column
ds_tweets['python'] = check_word_in_tweet('#python', ds_tweets)

# Create an rstats column
ds_tweets['rstats'] = check_word_in_tweet('#rstats', ds_tweets)

In [ ]:
# Average of python column by day
mean_python = ds_tweets['python'].resample('1 d').mean()

# Average of rstats column by day
mean_rstats = ds_tweets['rstats'].resample('1 d').mean()

# Plotting mean frequency

Lastly, we'll create a per-day average of the mentions of both hashtags and plot them across time. 
We'll first create proportions from the two boolean Series by the day, then we'll plot them.



In [ ]:
# Plot mean python by day(green)/mean rstats by day(blue)
plt.plot(mean_python.index.day, mean_python, color = 'green')
plt.plot(mean_rstats.index.day, mean_rstats, color = 'blue')

# Add labels and show
plt.xlabel('Day'); plt.ylabel('Frequency')
plt.title('Language mentions over time')
plt.legend(('#python', '#rstats'))
plt.show()

# Sentiment Analysis

Sentiment analysis is a type of natural language processing method which determines wheter a word, sentence, paragraph, or document is positive or negative. The idea behind sentiment analysis is that we count the words which are positive or negative as a proportion of the words in the rest of the document. Each document then gets a positive and negative score. Sentiment analysis can be useful in gauging reactions to a company, product, politician, or policy.

We use the VADER SentimentIntensityAnalyzer included in the Natural Language Toolkit or ‘nltk’. The VADER toolkit handles short text documents like tweets very well because it measures sentiment not only with particular words, but also for emoji and different type of capitalization.


Interpreting sentiment Scores

A critical part of any type of natural  language processing involves reading the text and assessing wheter the method makes sense compared to a human reading. If we’re attempting to replicate meaning with computational methods, then we have to make sure that meaning has face validity. 

Face validity means that the metric matches the concept we’re trying to measure. In this cases we want to be sure the sentiment score matches our idea what it means for a tweet to be positive or negative.

Each sentiment score from the VADER analyzer provides foru values: negative, neutral, positive, and compound. Compound is a combination of the positive and the negative. It’s a overall assessment which ranges between negative 1 and positive 1. Below 0 is negative and above 0 is positive.

Generating Sentiment Averages

We generate sentiment averages over time in the same way we generate average prevalence measure. We will extract only the ‘compound’ filed from the sentiment scores.

Next, we separate sentiment for each company with our ‘check_word_in_tweet’ function. Lastly we will generate an average value for our time window of one minute. 

We can plot sentiment scores in the same way we plotted the prevalence of keywords. We set the x-axis to time, and y-axis to our sentiment score.


# Loading VADER

Sentiment analysis provides us a small glimpse of the meaning of texts with a rather 
directly interpretable method. While it has its limitations, it's a good place to begin
working with textual data. There's a number of out-of-the-box tools in Python we can use 
for sentiment analysis.

In [ ]:
# Load SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate new SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Generate sentiment scores
sentiment_scores = ds_tweets['text'].apply(sid.polarity_scores)

# Calculating sentiment scores

A rough measure of sentiment towards a particular hashtag is to measure average sentiment
for tweets mentioning a particular hashtag. It's also possible that other things are happening 
in that tweet, so it's important to inspect both text as well as metrics generated by automated text methods.



In [ ]:
# Print out the text of a positive tweet
print(ds_tweets[sentiment > 0.6]['text'].values[0])

# Print out the text of a negative tweet
print(ds_tweets[sentiment < -0.6]['text'].values[0])

# Generate average sentiment scores for #python
sentiment_py = sentiment[ check_word_in_tweet('#python', ds_tweets) ].resample('1 d').mean()

# Generate average sentiment scores for #rstats
sentiment_r = sentiment[ check_word_in_tweet('#rstats', ds_tweets) ].resample('1 d').mean()

# Plotting sentiment scores

Lastly, let's plot the sentiment of each hashtag over time. 
This is largely similar to plotting the prevalence of tweets.

In [ ]:
# Import matplotlib
import matplotlib.pyplot as plt

# Plot average #python sentiment per day
plt.plot(sentiment_py.index.day, sentiment_py, color = 'green')

# Plot average #rstats sentiment per day
plt.plot(sentiment_r.index.day, sentiment_r, color = 'blue')

plt.xlabel('Day')
plt.ylabel('Sentiment')
plt.title('Sentiment of data science languages')
plt.legend(('#python', '#rstats'))
plt.show()

# Twitter Networks

We generate those networks using the ‘networkx’ and graph those networks. 

A ‘node’ is the actor within a network. Within tweeter the node are twitter user. 

An ‘edge’ or the ‘tie’ specifies the relationship between nodes.

We are going to look at multiple types of relationships. Here we will discuss Directed network, 
which means that an edge only goes on way – the relationship are not mutual. For example you can retweet someone else but they don’t have to retweet you. The source node is where the relationship begins, where the target node is where the relationship ends.

Retweets network: The retweet is a directed edge in which the source node retweets the target node. 
Retweets often signal agreement but not endorsement.



# Creating retweet network

Social media is, by nature, networked data. 
Twitter networks manifest in multiple ways.
One of the most important types of networks that appear in Twitter are retweet networks. 
We can represent these as directed graphs, with the retweeting user as the source and the 
retweeted person as the target. With Twitter data in our flattened DataFrame, we can import 
these into networkx and create a retweet network.

In [ ]:
# Import networkx
import networkx as nx

# Create retweet network from edgelist
G_rt = nx.from_pandas_edgelist(
    sotu_retweets,
    source = 'user-screen_name', 
    target = 'retweeted_status-user-screen_name',
    create_using = nx.DiGraph())
    
# Print the number of nodes
print('Nodes in RT network:', len(G_rt.nodes()))

# Print the number of edges
print('Edges in RT network:', len(G_rt.edges()))

# Creating reply network

Reply networks have a markedly different structure to retweet networks. While retweet networks often signal agreement, replies can signal discussion, deliberation, and disagreement. The network properties are the same, however: the network is directed, the source is the replier and the target is the user who is being replied to.



In [ ]:
import networkx as nx

# Create reply network from edgelist
G_reply = nx.from_pandas_edgelist(
    sotu_replies,
    source = 'user-screen_name',
    target = 'in_reply_to_screen_name',
    created_using = nx.DIGraph())
    
# Print the number of nodes
print('Nodes in reply network:', len(G_reply.nodes()))

# Print the number of edges
print('Edges in reply network:', len(G_reply.edges()))

# Visualizing retweet network

Visualizing retweets networks is an important exploratory data analysis step because it allows us to visually inspect the structure of the network, understand if there is any user that has disproportionate influence, and if there are different spheres of conversation.

In [ ]:
# Create random layout positions
pos = nx.random_layout(G_rt)

# Create size list
sizes = [x[1] for x in G_rt.degree()]

# Draw the network
nx.draw_networkx(G_rt, pos=pos, 
    with_labels = False, 
    node_size = sizes,
    width = 0.1, alpha = 0.7,
    arrowsize = 2, linewidths = 0)

# In-degree centrality

Centrality is a measure of importance of a node to a network. There are many different types of centrality and each of them has slightly different meaning in Twitter networks. We are first focusing on degree centrality, since its calculation is straightforward and has an intuitive explanation.

In [ ]:
# Generate in-degree centrality for retweets 
rt_centrality = nx.in_degree_centrality(G_rt)

# Generate in-degree centrality for replies 
reply_centrality = nx.in_degree_centrality(G_reply)

# Store centralities in DataFrame
rt = pd.DataFrame(list(rt_centrality.items()), columns = column_names)
reply = pd.DataFrame(list(reply_centrality.items()), columns = column_names)

# Print first five results in descending order of centrality
print(rt.sort_values('degree_centrality', ascending = False).head())

# Betweenness Centrality

Betweenness centrality for retweet and reply networks signals users who bridge between different Twitter communities. These communities may be tied together by topic or ideology.

In [ ]:
# Generate betweenness centrality for retweets 
rt_centrality = nx.betweenness_centrality(G_rt)

# Generate betweenness centrality for replies 
reply_centrality = nx.betweenness_centrality(G_reply)

# Store centralities in data frames
rt = pd.DataFrame(list(rt_centrality.items()), columns = column_names)
reply = pd.DataFrame(list(reply_centrality.items()), columns = column_names)

# Print first five results in descending order of centrality
print(rt.sort_values('betweenness_centrality', ascending = False).head())

# Ratios

While not strictly a measure of importance to a network, the idea of being "ratio'd" is a network measure which is particular to Twitter and is typically used to judge the unpopularity of a tweet. "The Ratio," as it is called, is calculated by taking the number of replies and dividing it by the number of retweets. For our purposes, it makes conceptual sense to take only the in-degrees of both the retweet and reply networks.

In [ ]:
# Calculate in-degrees and store in DataFrame
degree_rt = pd.DataFrame(list(G_rt.in_degree()), columns = column_names)
degree_reply = pd.DataFrame(list(G_reply.in_degree()), columns = column_names)

# Merge the two DataFrames on screen name
ratio = degree_rt.merge(degree_reply, on = 'screen_name', suffixes = ('_rt', '_reply'))

# Calculate the ratio
ratio['ratio'] = ratio['degree_reply'] / ratio['degree_rt']

# Exclude any tweets with less than 5 retweets
ratio = ratio[ratio['degree_rt'] >= 5]

# Print out first five with highest ratio
print(ratio.sort_values('ratio', ascending = False).head())

# Accessing user-defined location

In the slides, we saw that we could obtain user location via user-generated text, including the tweet itself and the location field in the user's description. These are the two most imprecise methods of obtaining user location, but also possibly more readily available.

In this exercise, you're going extract the user-defined location from a single example tweet as well as a large set of tweets. We've added another line to our flatten_tweets() function which will allow you to access user-defined location within the data frame.

In [ ]:
# Print out the location of a single tweet
print(tweet_json['user']['location'])

# Flatten and load the SOTU tweets into a dataframe
tweets_sotu = pd.DataFrame(flatten_tweets(tweets_sotu_json))

# Print out top five user-defined locations
print(tweets_sotu['user-location'].value_counts().head())

# Accessing bounding box

Most tweets which have coordinate-level geographical information attached to them typically come in the form of a bounding box. Bounding boxes are a set of four longitudinal/latitudinal coordinates which denote a particular area in which the user can be located. 
The bounding box is located in the place value of the Twitter JSON.

Geographical Data in Twitter JSON

A common – although imprecise – place where location data can be located is in the Twitter itself. Users mention or allude to a location where they are or have visited.

Once we have a location we need to resolve it into a latitude and longitude.

Another value in the Twitter JSON which offers location information is the user-defined location field. This is a text field which the user fills out. It can be found in the ‘location’ field in the Twitter user JSON.

The next value in which we can find geographical information is ‘place’. It is a child JSON object which contains several pieces of information. The most important of these is the ‘bounding_box’, which contains specific coordinates. The bounding box is a special kind of geographical object which allows Twitter to encode some uncertainty in location. As the box indicates it is a set of four longitude and latitude coordinates which create a box which surrounds the location. This is the most common type of geographical object we will find in Twitter data, so we will spend the most time with this. The ‘place’ object also includes country code, full and short name of the place, and the place type, along with some identification data. The bounding box can range from a city block to a whole state or even country. For simplicity’s sake one way we can deal with handling these data is by translating the bounding box into what’s called a centroid or, the center of the bounding box.


In [ ]:
def getBoundingBox(place):
    """ Returns the bounding box coordinates."""
    return place['bounding_box']['coordinates']

# Apply the function which gets bounding box coordinates
bounding_boxes = tweets_sotu['place'].apply(getBoundingBox)

# Print out the first bounding box coordinates
print(bounding_boxes.values[0])

# Calculating the centroid

The bounding box can range from a city block to a whole state or even country. For simplicity's sake, one way we can deal with handling these data is by translating the bounding box into what's called a centroid, or the center of the bounding box. The calculation of the centroid is straight forward -- we calculate the midpoints of the lines created by the latitude and longitudes.

In [ ]:
def calculateCentroid(place):
    """ Calculates the centroid from a bounding box."""
    # Obtain the coordinates from the bounding box.
    coordinates = place['bounding_box']['coordinates'][0]
    
    longs = np.unique( [x[0] for x in coordinates] )
    lats  = np.unique( [x[1] for x in coordinates] )
    
    if len(longs) == 1 and len(lats) == 1:
        # return a single coordinate
        return (longs[0], lats[0])
    elif len(longs) == 2 and len(lats) == 2:
        # If we have two longs and lats, we have a box.
        central_long = np.sum(longs) / 2
        central_lat  = np.sum(lats) / 2
    else:
        raise ValueError("Non-rectangular polygon not supported.")

    return (central_long, central_lat)

# Calculate the centroids of place     
centroids = tweets_sotu['place'].apply(calculateCentroid)

# Creating Twitter maps

Basemap is a library for plotting two dimensional maps in Python. Basemap is built on top of matplotlib so much of the functionality that you got out of matplotlib applies maps you created with it. Basemap transforms coordinates into map projections, while matplotlib draws contours lines and points.

A map projection is the method for which we translate latitudes and longitudes of the Earth – which is spherical – onto a two-dimensional plane. Because we will be dealing with latitudes closer to the equator, we will use the Mercator projection, which is denoted by ‘merc’.  We then provide coordinates for the lower lever and upper right corners of the map.


In [ ]:
# Import Basemap
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

# Set up the US bounding box
us_boundingbox = [-125, 22, -64, 50] 

# Set up the Basemap object
m = Basemap(llcrnrlon = us_boundingbox[0],
            llcrnrlat = us_boundingbox[1],
            urcrnrlon = us_boundingbox[2],
            urcrnrlat = us_boundingbox[3],
            projection='merc')

# Draw continents in white,
# coastlines and countries in gray
m.fillcontinents(color='white')
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')

# Draw the states and show the plot
m.drawstates(color='gray')
plt.show()

# Plotting centroid coordinates

Because we can't plot whole bounding boxes, we summarize the bounding box location into a single point called a centroid. Plotting these on a Basemap map is straightforward. Once we calculate the centroids, we separate the longitudes and latitudes, then pass to the .scatter() method.

In [ ]:
# Calculate the centroids for the dataset
# and isolate longitudue and latitudes
centroids = tweets_sotu['place'].apply(calculateCentroid)
lon = [x[0] for x in centroids]
lat = [x[1] for x in centroids]

# Draw continents, coastlines, countries, and states
m.fillcontinents(color='white', zorder = 0)
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# Draw the points and show the plot
m.scatter(lon, lat, latlon = True, alpha = 0.7)
plt.show()

# Coloring by sentiment

We want to be able to differentiate by place with our Twitter analysis. One distinguishing factor between places is how the State of the Union speech was received. For this purpose, we'll use the sentiment analysis we covered in Chapter 2 to evaluate how the speech was received in different parts of the country.

In [ ]:
# Generate sentiment scores
sentiment_scores = tweets_sotu['text'].apply(sid.polarity_scores)

# Isolate the compound element
sentiment_scores = [x['compound'] for x in sentiment_scores]

# Draw the points
m.scatter(lon, lat, latlon = True, 
           c = sentiment_scores,
           cmap = 'coolwarm', alpha = 0.7)

# Show the plot
plt.show()